In [1]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#   raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

# You'll generate plots of attention in order to see which parts of an image
# our model focuses on during captioning
import matplotlib.pyplot as plt

# Scikit-learn includes many helpful utilities
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import re
import numpy as np  
import os
import time
import json
from glob import glob
from PIL import Image
import pickle
import cv2
from sklearn.utils import shuffle

print(tf.__version__) 


Found GPU at: 
2.2.0


In [2]:
from IPython.display import clear_output, display
import os.path
from os import path
import shutil
import PIL
from pathlib import Path
from matplotlib import pyplot as plt

In [7]:
import random

In [ ]:
BASE_DIR= '/mestrado/datasets'
IMG_ORG_DIR= path.join( BASE_DIR, 'carnaval_8lines_shuffle_parts', 'images')
LABELS_ORG_DIR= path.join( BASE_DIR, 'carnaval_8lines_shuffle_parts', 'labels')

In [3]:
def read_label( path):
  f= open( path)
  d= f.read()
  f.close()
  return d 

def write_label( path, text):
  f= open( path, "w")
  f.write( text)
  f.close()

def show_image( img, name= 'noname'):
  plt.imshow( img)
  plt.title( name)
  plt.show()

def show_gimage( img, name= 'noname'):
  plt.imshow( img, cmap= 'gray')
  plt.title( name)
  plt.show()

def _load_image( path):
  # return cv2.imread( path, cv2.IMREAD_GRAYSCALE)
  return cv2.imread( path)
  
def write_image( path, img):
  cv2.imwrite( path, img)

def resize( img, scale_percent= 60, verbose=False):
  #scale_percent = 60 # percent of original size
  width = int(img.shape[1] * scale_percent / 100)
  height = int(img.shape[0] * scale_percent / 100)
  dim = (width, height)
  if verbose:
    print( "scale", scale_percent)
    print( "dim", dim)
  # resize image
  return cv2.resize(img, dim, interpolation = cv2.INTER_AREA) 

In [4]:
def _load_all():
    all_images= glob( os.path.join( IMG_ORG_DIR, "*.jpg"))
    # all_images= glob( '/content/carnaval_8lines/images/*.jpg')
    all_images.sort()
#     all_images= all_images[0:limit]
    all_images_content= [_load_image( f) for f in all_images]    
    all_images_content= np.asarray( all_images_content)

    all_labels=glob( os.path.join( LABELS_ORG_DIR, "*.pgn"))
    all_labels.sort()
#     all_labels= all_labels[0:limit]

    #ler todo o conteudo de labels e criar o vocabulario
    all_labels_content= [read_label( f) for f in all_labels]
#     all_labels_content= [label.split() for label in all_labels_content]

    # pega somente os primeiros 4, equivalente a 1 poagina
#     all_labels_content= [x[0:16] for x in all_labels_content]

    #complementa onde falta para completar 100
#     all_labels_content= [
#                 np.pad( x, (0,16-len(x)), 'constant', constant_values= 'end') for x in all_labels_content]
    return all_images, all_images_content, all_labels, all_labels_content


def cut( img, lines, cols):
    cuts= []
    h= (int)(img.shape[ 0]/lines)
    w= (int)(img.shape[ 1]/cols)

    for i in range(0,lines):
      for j in range(0,cols):
        cuts.append( img[ (i*h):(i+1)*h, (j*w):(j+1)*w])
    return np.array( cuts)

def rebuild( cuts):
    return np.concatenate( 
        (np.concatenate( (cuts[0], cuts[2], cuts[4], cuts[6], cuts[8], cuts[10], cuts[12], cuts[14]), axis=0),
        np.concatenate( (cuts[1], cuts[3], cuts[5], cuts[7], cuts[9], cuts[11], cuts[13], cuts[15]), axis=0)),
        axis= 1
    )

def _shuffle( img, label, k):
    cuts= cut( img, 8, 2)

    # newcuts, newlabel= np.random.shuffle( cuts)
    newcuts, newlabel= shuffle( cuts, label, random_state= k)
    newimg= rebuild( newcuts)
    return newimg, newlabel


In [ ]:
all_images, all_image_content, all_labels_files, label_content= _load_all()

In [ ]:
print( label_content[0])
show_image( all_image_content[0])

In [ ]:
len( label_content)

In [ ]:
# IMG_TARGET_DIR= path.join( BASE_DIR, 'carnaval_8lines_shuffle_from_parts20k', 'images')
# LABELS_TARGET_DIR= path.join( BASE_DIR, 'carnaval_8lines_shuffle_from_parts20k', 'labels')


# all_images, all_image_content, all_labels_files, label_content= _load_all()


# new_cuts= all_image_content
# new_labels= label_content

# size= len( new_cuts)
# def generate(N):
#     count= 0;
    
#     for i in range( 0, N):
#         id= [np.random.randint(0, size) for _ in range( 0, 16)]
        
#         img= rebuild( [new_cuts[j] for j in id])
#         label= [new_labels[j] for j in id]
#         tag= "shuffled_"+str(i+1).zfill(5) 
#         write_image( os.path.join( IMG_TARGET_DIR, tag+ ".jpg"), resize( img, 50))
#         write_label( os.path.join( LABELS_TARGET_DIR, tag+ ".pgn"), " ".join( label))
#         if i%500== 0:
#             print( i)
            
# generate(20000)
# print( "DONE")

In [ ]:
# all_labels=glob( os.path.join( LABELS_ORG_DIR, "*.pgn"))
# all_labels_content= [read_label( f) for f in all_labels]


In [ ]:
# labels= all_labels_content.sort()

In [ ]:
# https://github.com/mdbloice/Augmentor
samples_dir= "/mestrado/datasets/parts_sample"
# samples_dir= "C:/mestrado/datasets/carnaval_8lines_shuffle_parts/images"

In [2]:
import Augmentor


In [10]:
p = Augmentor.Pipeline('C:\mestrado\datasets\carnaval_8lines_shuffle_from_parts_50k\images')
p.random_distortion(probability=0.8, grid_width=4, grid_height=8, magnitude=8)
# p.zoom(probability=0.3, min_factor=1.1, max_factor=1.6)
# p.rotate(probability=0.5, max_left_rotation=10, max_right_rotation=10)
# p.flip_left_right(probability=0.5)
# p.flip_top_bottom(probability=0.5)
p.sample(50000)

Executing Pipeline:   0%|                                                              | 0/50000 [00:00<?, ? Samples/s]

Initialised with 50000 image(s) found.
Output directory set to C:\mestrado\datasets\carnaval_8lines_shuffle_from_parts_50k\images\output.

Processing <PIL.Image.Image image mode=RGB size=339x288 at 0x2E9ED8C7A00>: 100%|█| 50000/50000 [05:02<00:00, 165.22 Sam


In [ ]:
path.basename( images[0])

In [ ]:
label= path.basename( images[0])[27:32].replace( '_', '')
print( label)

In [ ]:
fname= str(seq)
"shuffled_"+str(i+1).zfill(5) 
#         write_image( os.path.join( IMG_TARGET_DIR, tag+ ".jpg"), resize( img, 50))
#         write_label( os.path.join( LABELS_TARGET_DIR, tag+ ".pgn"), " ".join( label))

In [14]:
def getlabel( file):
    return path.basename( file)[27:32].replace( '_', '')

In [8]:
#
# Gera shuffle a partir das imagens augmentadas
#
IMG_TARGET_DIR= 'C:\mestrado\datasets\carnaval_8lines_shuffle_from_parts_augmented\images'
LABELS_TARGET_DIR= 'C:\mestrado\datasets\carnaval_8lines_shuffle_from_parts_augmented\labels'

all_images= glob( "C:\mestrado\datasets\carnaval_8lines_shuffle_parts_augmented\images\*.jpg")

In [11]:
# all_images= glob( '/content/carnaval_8lines/images/*.jpg')
random.shuffle(all_images)